In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random as random
%matplotlib inline
from casadi import *
from pdfo import *
import dataframe_image as dfi

In [2]:
n = 500
taux = 1/n

In [3]:
############################################# NON-HOLONOMIQUE 


####################################### KKT

####################### DOC 

def Unicycle_DOC ( Xi , Xf , c1 , c2) :
    x1i = Xi[0] 
    x2i = Xi[1]
    x3i = Xi[2]
    
    x1f = Xf[0] 
    x2f = Xf[1]
    x3f = Xf[2]
    
    opti = casadi.Opti()   # cette fonction nous permet de trouver la solution de problème

    u1 = opti.variable(n)
    u2 = opti.variable(n)
    x1 = opti.variable(n)
    x2 = opti.variable(n)
    x3 = opti.variable(n)

    opti.minimize(   (taux/2)*(c1*dot(u1,u1)+c2*dot(u2,u2))   )    # ma fonction objetion

    # mes fonctions de contrainte d'égalité:

    ## pour les condition initial
    opti.subject_to( x1[0]==x1i + 10**(-4))        
    opti.subject_to( x2[0]==x2i + 10**(-4))
    opti.subject_to( x3[0]==x3i + 10**(-4))
    
    opti.subject_to( u1[0] == 0.0001 )
    opti.subject_to( u2[0] == 0.0001 )

    opti.subject_to( u1[-1] == 0.0001)
    opti.subject_to( u2[-1] == 0.0001)

    ## pour les contraintes d'égaliter
    opti.subject_to( x1[:n-1]+taux*u1[:n-1]*cos(x3[:n-1])==x1[1:] + 10**(-4))
    opti.subject_to( x2[:n-1]+taux*u1[:n-1]*sin(x3[:n-1])==x2[1:] + 10**(-4))
    opti.subject_to( x3[:n-1]+taux*u2[:n-1] ==x3[1:] + 10**(-4))

    ## pour les conditions finales
    opti.subject_to( x1[-1]==x1f + 10**(-4))
    opti.subject_to( x2[-1]==x2f + 10**(-4))
    opti.subject_to( x3[-1]==x3f + 10**(-4))


    opti.solver('ipopt')      # suivant la méthode de KKT


    sol = opti.solve()
    
    X1 = sol.value(x1)
    X2 = sol.value(x2)
    X3 = sol.value(x3)
    
    U1 = sol.value(u1)
    U2 = sol.value(u2)
    
    return X1,X2,X3,U1,U2

#################################### IOC

x1i = SX.sym('x1i',1)                   
x2i = SX.sym('x2i',1)                
x3i = SX.sym('x3i',1)


x1f = SX.sym('x1f',1)
x2f = SX.sym('x2f',1)
x3f = SX.sym('x3f',1)


c1 = SX.sym('c1',1)
c2 = SX.sym('c2',1)

u1=SX.sym('u1',n)  
u1_prime = SX.sym('u1_prime', n+1)
u1_prime[0] = 0
u1_prime[n] = 0
u1_prime[1:n] =u1[0:n-1]

u2=SX.sym('u2',n)  
u2_prime = SX.sym('u2_prime', n+1)
u2_prime[0] = 0
u2_prime[n] = 0
u2_prime[1:n] =u2[0:n-1]

x1=SX.sym('x1',n)
x1_prime = SX.sym('x1_prime', n+1)
x1_prime[0] = x1[0]
x1_prime[1:] =x1


x2=SX.sym('x2',n)
x2_prime = SX.sym('x1_prime', n+1)
x2_prime[0] = x2[0]
x2_prime[1:] =x2

x3=SX.sym('x3',n)
x3_prime = SX.sym('x1_prime', n+1)
x3_prime[0] = x3[0]
x3_prime[1:] =x3

Lambda = SX.sym('Lambda',n+2, 3)
Mue = SX.sym('Lambda',1)

p1=vertcat(x1i + 10**(-4),x1_prime[2:] + 10**(-4),x1f + 10**(-4))   # Je définis un nouveau vecteur suivant x1 en prenant les n-1 dernières valeurs 
                               #  et la nième valeur vaut x1f
g1= Function('g1',[x1, x1i , x1f],[p1])

Y1_U = (x1_prime+taux*u1_prime*cos(x3_prime) - g1(x1, x1i,x1f))
Y2_U = (x2_prime+taux*u1_prime*sin(x3_prime) - g1(x2, x2i,x2f)) 
Y3_U = (x3_prime+taux*u2_prime - g1(x3, x3i,x3f))
Y_U = SX.sym('Y',n+1 , 3)        ## notre contrainte

for i in range (n+1):
    Y_U[i,0]= Y1_U[i]
    Y_U[i,1]= Y2_U[i]
    Y_U[i,2]= Y3_U[i] 
    
    
    
## notre terme qui est relié a la contrainte.
G_lambda = 0

for i in range (n+1):
    G_lambda += dot(Y_U[i,:], Lambda[i,:])
    
G_lambda += (u1[0]-0.0001)*Lambda[n+1,0] + (u2[0]-0.0001)*Lambda[n+1,1] + (u1[-1]-0.0001)*Lambda[n+1,2] + (u2[-1]-0.0001)*Mue


    
G_U = Function('G_U', [x1,x2,x3, Lambda], [G_lambda])


## notre fonction F 
F_val_U = (taux/2)*(c1*dot(u1,u1)+c2*dot(u2,u2))


## le Lagrangien 
L_val_U = F_val_U + G_lambda


grad_L_U = SX.zeros(5, n)
for i in range (n):
    grad_L_U[0,i]= jacobian(L_val_U, u1[i])
    grad_L_U[1,i]= jacobian(L_val_U, u2[i])
    grad_L_U[2,i]= jacobian(L_val_U, x1[i])
    grad_L_U[3,i]= jacobian(L_val_U, x2[i])
    grad_L_U[4,i]= jacobian(L_val_U, x3[i])
    
    
R_U = Function ('R_U', [u1,u2,x1,x2,x3, Lambda,Mue, c1, c2, x1i,x2i,x3i, x1f,x2f,x3f ], [(dot(grad_L_U,grad_L_U))])


def Unicycle_IOC (R_U, U1,U2,X1,X2,X3,Xi,Xf) :
    x1i = Xi[0] 
    x2i = Xi[1]
    x3i = Xi[2]
    
    x1f = Xf[0] 
    x2f = Xf[1]
    x3f = Xf[2]
    
    opti = casadi.Opti()   # cette fonction nous permet de trouver la solution de problème

    c1    = opti.variable()
    c2    = opti.variable()

    Lambda = opti.variable(n+2,3)
    Mue = opti.variable(1)
    
    
    opti.minimize( R_U(U1,U2,X1,X2,X3, Lambda, Mue, c1, c2 , x1i,x2i,x3i, x1f,x2f,x3f ))  

    opti.subject_to( 0 <= c1)
    opti.subject_to( 0 <= c2 )
    opti.subject_to(  c1 + c2 == 1)

    opti.solver('ipopt')    

    sol = opti.solve()
    
    return sol.value(c1), sol.value(c2) , sol.value(Lambda), sol.value(Mue)

############################################# BL1

X1=SX.sym('X1',n)
X2=SX.sym('X2',n)  
X3=SX.sym('X3',n)  
m = SX.sym('m',1)
m = (dot(X1-x1,X1-x1) + dot(X2-x2,X2-x2) + dot(X3-x3,X3-x3))

M = Function ('M', [x1,x2,x3, X1,X2,X3], [m])

def BL (U1,U2,X1,X2,X3, C1,C2, Xi, Xf):
    opti = casadi.Opti()   

    c1 = opti.variable(1)
    c2 = opti.variable(1)
    Lambda = opti.variable(n+2,3)
    Mue = opti.variable(1)
    u1 = opti.variable(n)
    u2 = opti.variable(n)
    x1 = opti.variable(n)
    x2 = opti.variable(n)
    x3 = opti.variable(n)


    opti.minimize((M(x1,x2,x3, X1,X2,X3)) )  

    # mes fonctions de contrainte d'égalité:
    opti.subject_to( 0 <= c1)
    opti.subject_to( 0 <= c2 )
    opti.subject_to(  c1 + c2 == 1)
    
    ## pour les condition initial
    opti.subject_to( x1[0]==Xi[0] + 10**(-4))        
    opti.subject_to( x2[0]==Xi[1] + 10**(-4))
    opti.subject_to( x3[0]==Xi[2] + 10**(-4))

    opti.subject_to( u1[0] == 0.0001 )
    opti.subject_to( u2[0] == 0.0001 )
    opti.subject_to( u1[-1] == 0.0001)
    opti.subject_to( u2[-1] == 0.0001)
    
    

    ## pour les contraintes d'égaliter
    opti.subject_to(R_U(u1,u2,x1,x2,x3, Lambda, Mue, c1, c2 , X1[0],X2[0],X3[0], X1[-1],X2[-1],X3[-1] ) <= 10**(-6) )
    opti.subject_to( u1[:n-1]*cos(x3[:n-1])==(x1[1:] + 10**(-4) - x1[:n-1])/taux )
    opti.subject_to( u1[:n-1]*sin(x3[:n-1])==(x2[1:] + 10**(-4) - x2[:n-1])/taux )
    opti.subject_to( u2[:n-1] ==(x3[1:] + 10**(-4) - x3[:n-1])/taux)


    ## pour les conditions finales
    opti.subject_to( x1[-1]==Xf[0] + 10**(-4))
    opti.subject_to( x2[-1]==Xf[1] + 10**(-4))
    opti.subject_to( x3[-1]==Xf[2] + 10**(-4))
    
    opti.set_initial(c1, C1)
    opti.set_initial(c2, C2)
    
    opti.set_initial(u1, U1)
    opti.set_initial(u2, U2)
    opti.set_initial(x1, X1)
    opti.set_initial(x2, X2)
    opti.set_initial(x3, X3)
    

    opti.solver('ipopt')      


    sol = opti.solve()
    
    return sol.value(c1), sol.value(c2), sol.value(x1), sol.value(x2), sol.value(x3)


########################################## PDFO

bounds = np.array([[0, 1], [0, 1]])
lin_con = LinearConstraint([1, 1], 1, 1)

options = {'maxfev': 300 , 'rhobeg' : 0.01 , 'rhoend' : 1e-6}


def Unicycle (C) :
    [C1,C2] = C
    print(C)
    
    #if C1 < 0 or C2 < 0 or C1 > 1 or C2 > 1 or C1 + C2 != 1 :
     #   return 500 
    
    opti = casadi.Opti()   # cette fonction nous permet de trouver la solution de problème

    u1 = opti.variable(n)
    u2 = opti.variable(n)
    x1 = opti.variable(n)
    x2 = opti.variable(n)
    x3 = opti.variable(n)

    opti.minimize(   (taux/2)*(C1*dot(u1,u1)+C2*dot(u2,u2))   )    # ma fonction objetion

    # mes fonctions de contrainte d'égalité:

    ## pour les condition initial
    opti.subject_to( x1[0]==X1[0] + 10**(-4))        
    opti.subject_to( x2[0]==X2[0] + 10**(-4))
    opti.subject_to( x3[0]==X3[0] + 10**(-4))
    
    opti.subject_to( u1[0] == 0.0001 )
    opti.subject_to( u2[0] == 0.0001 )
    opti.subject_to( u1[-1] == 0.0001)
    opti.subject_to( u2[-1] == 0.0001)
    
    for j in range (n):
        opti.subject_to( u1[j] < 10 )
        opti.subject_to( u1[j] > -10 )
        
        opti.subject_to( u2[j] < 20 )
        opti.subject_to( u2[j] > -20 )
        
    

    ## pour les contraintes d'égaliter
    opti.subject_to( u1[:n-1]*cos(x3[:n-1])==(x1[1:] + 10**(-4) - x1[:n-1])/taux)
    opti.subject_to( u1[:n-1]*sin(x3[:n-1])==(x2[1:] + 10**(-4) - x2[:n-1])/taux)
    opti.subject_to( u2[:n-1] ==(x3[1:] + 10**(-4) - x3[:n-1])/taux)
    
    ## pour les conditions finales
    opti.subject_to( x1[-1]==X1[-1] + 10**(-4))
    opti.subject_to( x2[-1]==X2[-1] + 10**(-4))
    opti.subject_to( x3[-1]==X3[-1] + 10**(-4))
    
    opti.solver('ipopt', {"expand":True}, {"max_iter":10000})
    
    sol = opti.solve() 
    
    X1_1 = opti.debug.value(x1)
    X2_1 = opti.debug.value(x2)
    X3_1 = opti.debug.value(x3)
    
    plt.plot(X1_1,X2_1, color = 'green')
    
    m1 = sqrt((np.linalg.norm(X1-X1_1)**2 + np.linalg.norm(X2-X2_1)**2 + np.linalg.norm(X3-X3_1)**2 )/n)
    
    return m1   


def DOC_Unicycle_PDFO (Xi, Xf, C1,C2):

    opti = casadi.Opti()   # cette fonction nous permet de trouver la solution de problème

    u1 = opti.variable(n)
    u2 = opti.variable(n)
    x1 = opti.variable(n)
    x2 = opti.variable(n)
    x3 = opti.variable(n)

    opti.minimize(   (taux/2)*(C1*dot(u1,u1)+C2*dot(u2,u2))   )    # ma fonction objetion

        # mes fonctions de contrainte d'égalité:

        ## pour les condition initial
    opti.subject_to( x1[0]==Xi[0] + 10**(-4))        
    opti.subject_to( x2[0]==Xi[1] + 10**(-4))
    opti.subject_to( x3[0]==Xi[2] + 10**(-4))

    opti.subject_to( u1[0] == 0.0001 )
    opti.subject_to( u2[0] == 0.0001 )
    opti.subject_to( u1[-1] == 0.0001)
    opti.subject_to( u2[-1] == 0.0001)

    for j in range (n):
        opti.subject_to( u1[j] < 10 )
        opti.subject_to( u1[j] > -10 )

        opti.subject_to( u2[j] < 20 )
        opti.subject_to( u2[j] > -20 )



        ## pour les contraintes d'égaliter
    opti.subject_to( u1[:n-1]*cos(x3[:n-1])==(x1[1:] + 10**(-4) - x1[:n-1])/taux)
    opti.subject_to( u1[:n-1]*sin(x3[:n-1])==(x2[1:] + 10**(-4) - x2[:n-1])/taux)
    opti.subject_to( u2[:n-1] ==(x3[1:] + 10**(-4) - x3[:n-1])/taux)

        ## pour les conditions finales
    opti.subject_to( x1[-1]==Xf[0] + 10**(-4))
    opti.subject_to( x2[-1]==Xf[1] + 10**(-4))
    opti.subject_to( x3[-1]==Xf[2] + 10**(-4))

    opti.solver('ipopt', {"expand":True}, {"max_iter":10000})

    sol = opti.solve() 

    X01 = opti.debug.value(x1)
    X02 = opti.debug.value(x2)
    X03 = opti.debug.value(x3)
    
    
    return X01,X02,X03


In [4]:
############################################# HOLONOMIQUE



####################################### KKT

######################## DOC

def Katja_Mombaur_DOC ( Xi, Xf, alpha1, alpha2, alpha3):
    xi = Xi[0] 
    yi = Xi[1]
    thetai = Xi[2]
    
    xf = Xf[0] 
    yf = Xf[1]
    thetaf = Xf[2]
    
    opti = casadi.Opti()   

    ## les positions
    x = opti.variable(n)
    y = opti.variable(n)
    theta = opti.variable(n)

    ## les vitesses 
    v1 = opti.variable(n)        ## vitesse latérale
    v2 = opti.variable(n)        ## vitesse orthogonal
    w = opti.variable(n)         ## vitesse angulaire
    
        ## les vitesses 
    u1 = opti.variable(n)        ## accélération latérale
    u3 = opti.variable(n)        ## accélération orthogonal
    u2 = opti.variable(n)        ## accélération angulaire
    
    
    
    opti.minimize(  taux*(alpha1 * dot(u1,u1) + alpha2 * dot(u2,u2 ) + alpha3 * dot( u3 ,u3 ) ) )    # ma fonction objetion

    # mes fonctions de contrainte d'égalité:

    ## pour les condition initial
    opti.subject_to( x[0] == xi + 10**(-4))        
    opti.subject_to( y[0] == yi + 10**(-4))
    opti.subject_to( theta[0] == thetai + 10**(-4))
    
    
    opti.subject_to( v1[0] == 0.0001 )
    opti.subject_to( w[0]  == 0.0001 )
    opti.subject_to( v2[0] == 0.0001 )
    opti.subject_to( v1[-1] == 0.0001 )
    opti.subject_to( w[-1]  == 0.0001 )
    opti.subject_to( v2[-1] == 0.0001 )
    
    opti.subject_to( u1[-1] == 0.0001 )
    opti.subject_to( u2[-1] == 0.0001 )
    opti.subject_to( u3[-1] == 0.0001 )
    opti.subject_to( u1[0] == 0.0001 )
    opti.subject_to( u2[0] == 0.0001 )
    opti.subject_to( u3[0] == 0.0001 )



    ## pour les contraintes d'égaliter
    opti.subject_to( x[1:] + 10**(-4) == x[:n-1]+taux*(cos(theta[:n-1])*v1[:n-1] - sin(theta[:n-1])*v2[:n-1]) )
    opti.subject_to( y[1:] + 10**(-4) == y[:n-1]+taux*(sin(theta[:n-1])*v1[:n-1] + cos(theta[:n-1])*v2[:n-1]) )
    opti.subject_to( theta[1:] + 10**(-4) == theta[:n-1] + taux*w[:n-1] )
    opti.subject_to( (v1[:n-1] + taux* u1[:n-1] == v1[1:] + 10**(-4))  )
    opti.subject_to( (v2[:n-1] + taux* u3[:n-1] == v2[1:] + 10**(-4)) )
    opti.subject_to( (w[:n-1] + taux* u2[:n-1] == w[1:] + 10**(-4)) )
    

    ## pour les conditions finales
    opti.subject_to( x[-1]==xf + 10**(-4))
    opti.subject_to( y[-1]==yf + 10**(-4))
    opti.subject_to( theta[-1]==thetaf + 10**(-4))


    opti.solver('ipopt')      


    sol = opti.solve()
    
    
    X = sol.value(x)
    Y = sol.value(y)
    THETA = sol.value(theta)
    
    V1 = sol.value(v1)
    V2 = sol.value(v2)
    W = sol.value(w)
    
    U1 = sol.value(u1)
    U2 = sol.value(u2)
    U3 = sol.value(u3)
    
    return X,Y,THETA, V1,V2,W, U1,U2,U3


####################### IOC


xi = SX.sym('xi',1)                   
yi = SX.sym('yi',1)                
thetai = SX.sym('thetai',1)


xf = SX.sym('xf',1)
yf = SX.sym('yf',1)
thetaf = SX.sym('thetaf',1)

alpha1 = SX.sym('alpha1',1)
alpha2 = SX.sym('alpha2',1)
alpha3 = SX.sym('alpha3',1)

## Position
x=SX.sym('x',n)
x_prime = SX.sym('x_prime', n+1)
x_prime[0] = x[0]
x_prime[1:] =x


y=SX.sym('y',n)
y_prime = SX.sym('y_prime', n+1)
y_prime[0] = y[0]
y_prime[1:] =y

theta=SX.sym('theta',n)
theta_prime = SX.sym('theta_prime', n+1)
theta_prime[0] = theta[0]
theta_prime[1:] =theta

## Vitesse
v1=SX.sym('v1',n)  
v1_prime = SX.sym('v1_prime', n+1)
v1_prime[0] = 0
v1_prime[n] = 0
v1_prime[1:n] =v1[0:n-1]

v1_prime_1 = SX.sym('v1_prime_1', n+1)
v1_prime_1[0] = v1[0]
v1_prime_1[1:] =v1


v2=SX.sym('v2',n)  
v2_prime = SX.sym('v2_prime', n+1)
v2_prime[0] = 0
v2_prime[n] = 0
v2_prime[1:n] =v2[0:n-1]

v2_prime_1 = SX.sym('v2_prime_1', n+1)
v2_prime_1[0] = v2[0]
v2_prime_1[1:] =v2


w=SX.sym('w',n)  
w_prime = SX.sym('w_prime', n+1)
w_prime[0] = 0
w_prime[n] = 0
w_prime[1:n] =w[0:n-1]

w_prime_1 = SX.sym('w_prime_1', n+1)
w_prime_1[0] = w[0]
w_prime_1[1:] =w

## Accélération 

u1=SX.sym('u1',n)  
u1_prime = SX.sym('u1_prime', n+1)
u1_prime[0] = 0
u1_prime[n] = 0
u1_prime[1:n] = u1[0:n-1]

u2=SX.sym('u2',n)  
u2_prime = SX.sym('u2_prime', n+1)
u2_prime[0] = 0
u2_prime[n] = 0
u2_prime[1:n] = u2[0:n-1]

u3=SX.sym('u3',n)  
u3_prime = SX.sym('u3_prime', n+1)
u3_prime[0] = 0
u3_prime[n] = 0
u3_prime[1:n] = u3[0:n-1]

Lambda = SX.sym('Lambda',n+3, 6)


p1=vertcat(xi + 10**(-4) ,x_prime[2:] + 10**(-4) ,xf + 10**(-4) )   
h= Function('h',[x, xi, xf],[p1])
p2=vertcat(0, v1)   
K = Function('K', [v1], [p2])
p =vertcat(v1[1:],0) + 10**(-4)
g = Function ('g',[v1],[p])


Y1_K = (x_prime+taux*(v1_prime*cos(theta_prime) - v2_prime*sin(theta_prime)) - h(x, xi,xf))
Y2_K = (y_prime+taux*(v1_prime*sin(theta_prime) + v2_prime*cos(theta_prime)) - h(y, yi,yf)) 
Y3_K = (theta_prime+taux*w_prime - h(theta, thetai,thetaf))

U1 = (g(v1)-v1)/taux - u1
U2 = (g(w)-w)/taux  - u2
U3 = (g(v2)-v2)/taux  - u3 

Y4_K = K(U1) 
Y5_K = K(U2)
Y6_K = K(U3)



Y_K = SX.sym('Y_K',n+1 , 6)        ## notre contrainte

for i in range (0,n+1):
    Y_K[i,0]= Y1_K[i]
    Y_K[i,1]= Y2_K[i]
    Y_K[i,2]= Y3_K[i]       
    Y_K[i,3]= Y4_K[i]       
    Y_K[i,4]= Y5_K[i]       
    Y_K[i,5]= Y6_K[i]       
    
## notre terme qui est relié a la contrainte.
G_lambda = 0

for i in range (n+1):
    G_lambda += dot(Y_K[i,:], Lambda[i,:])
    
G_lambda += (v1[0]-0.0001)*Lambda[n+1,0] + (w[0]-0.0001)*Lambda[n+1,1] + (v2[0]-0.0001)*Lambda[n+1,2] 
G_lambda += (v1[-1]-0.0001)*Lambda[n+1,3] + (w[-1]-0.0001)*Lambda[n+1,4] + (v2[-1]-0.0001)*Lambda[n+1,5] 

G_lambda += (u1[0]-0.0001)*Lambda[n+2,0] + (u2[0]-0.0001)*Lambda[n+2,1] + (u3[0]-0.0001)*Lambda[n+2,2] 
G_lambda += (u1[-1]-0.0001)*Lambda[n+2,3] + (u2[-1]-0.0001)*Lambda[n+2,4] + (u3[-1]-0.0001)*Lambda[n+2,5] 


F_val_K =  taux*( alpha1 * dot(u1,u1) + alpha2 * dot(u2,u2) + alpha3 * dot(u3,u3))

## le Lagrangien 
L_val_K = F_val_K + G_lambda

grad_L_K = SX.zeros(9, n)
for i in range (n):
    grad_L_K[0,i]= jacobian(L_val_K, v1[i])
    grad_L_K[1,i]= jacobian(L_val_K, w[i])
    grad_L_K[2,i]= jacobian(L_val_K, v2[i])
    grad_L_K[3,i]= jacobian(L_val_K, x[i])
    grad_L_K[4,i]= jacobian(L_val_K, y[i])
    grad_L_K[5,i]= jacobian(L_val_K, theta[i])
    grad_L_K[6,i]= jacobian(L_val_K, u1[i])
    grad_L_K[7,i]= jacobian(L_val_K, u2[i])
    grad_L_K[8,i]= jacobian(L_val_K, u3[i])
    
    
    
R_K = Function ('R_K', [u1,u2,u3,v1,w,v2,x,y,theta, Lambda, alpha1, alpha2, alpha3 ,xi,yi,thetai, xf,yf,thetaf  ], [dot(grad_L_K,grad_L_K)])




def Katja_Mombaur_IOC (R_K,U1,U2,U3 ,V1,V2,W,X,Y,THETA,Xi,Xf) :
    xi = Xi[0] 
    yi = Xi[1]
    thetai = Xi[2]
    
    xf = Xf[0] 
    yf = Xf[1]
    thetaf = Xf[2]
    
    
    opti = casadi.Opti()   # cette fonction nous permet de trouver la solution de problème

    alpha1 = opti.variable()
    alpha2 = opti.variable()
    alpha3 = opti.variable()
    Lambda = opti.variable(n+3,6)

    
    
    opti.minimize( R_K(U1,U2,U3,V1,W,V2,X,Y,THETA, Lambda, alpha1, alpha2, alpha3, xi,yi,thetai, xf,yf,thetaf  )) 

    opti.subject_to( 0 <= alpha1)
    opti.subject_to( 0 <= alpha2 )
    opti.subject_to( 0 <= alpha3 )
    opti.subject_to(  alpha1 + alpha2 + alpha3 == 1)

    opti.solver('ipopt')    

    sol = opti.solve()
    
    Alpha1 = sol.value (alpha1)
    Alpha2 = sol.value (alpha2)   
    Alpha3 = sol.value (alpha3)
    
    return Alpha1, Alpha2, Alpha3 , sol.value(Lambda)


################################################# BL1

X1=SX.sym('X1',n)
X2=SX.sym('X2',n)  
X3=SX.sym('X3',n)  


m = SX.sym('m',1)
m = (dot(X1-x,X1-x) + dot(X2-y,X2-y) + dot(X3-theta,X3-theta))

M = Function ('M', [x,y,theta, X1,X2,X3], [m])


def BL_Mombaur (U1,U2,U3 ,V1,W,V2,X,Y,THETA, Alpha1,Alpha2,Alpha3):
    opti = casadi.Opti()   # cette fonction nous permet de trouver la solution de problème

    alpha1 = opti.variable()
    alpha2 = opti.variable()
    alpha3 = opti.variable()
    Lambda = opti.variable(n+3,6)
    ## les positions
    x = opti.variable(n)
    y = opti.variable(n)
    theta = opti.variable(n)

    ## les vitesses 
    v1 = opti.variable(n)        ## vitesse latérale
    v2 = opti.variable(n)        ## vitesse orthogonal
    w = opti.variable(n)         ## vitesse angulaire


    ## les accélération 
    u1 = opti.variable(n)        ## accélération latérale
    u3 = opti.variable(n)        ## accélération orthogonal
    u2 = opti.variable(n)        ## accélération angulaire
    
    opti.minimize((M(x,y,theta, X,Y,THETA)) ) 
 

    # mes fonctions de contrainte d'égalité:
    opti.subject_to( 0 <= alpha1)
    opti.subject_to( 0 <= alpha2 )
    opti.subject_to( 0 <= alpha3 )
    opti.subject_to(  alpha1 + alpha2 + alpha3 == 1)
    
    opti.subject_to( x[0] == X[0] + 10**(-4))        
    opti.subject_to( y[0] == Y[0] + 10**(-4))
    opti.subject_to( theta[0] == THETA[0] + 10**(-4))
    
    opti.subject_to( v1[0] == 0.0001 )
    opti.subject_to( w[0]  == 0.0001 )
    opti.subject_to( v2[0] == 0.0001 )
    opti.subject_to( v1[-1] == 0.0001 )
    opti.subject_to( w[-1]  == 0.0001 )
    opti.subject_to( v2[-1] == 0.0001 )
        
    opti.subject_to( u1[-1] == 0.0001 )
    opti.subject_to( u2[-1] == 0.0001 )
    opti.subject_to( u3[-1] == 0.0001 )
    opti.subject_to( u1[0] == 0.0001 )
    opti.subject_to( u2[0] == 0.0001 )
    opti.subject_to( u3[0] == 0.0001 )



    ## pour les contraintes d'égaliter
    opti.subject_to(  R_K(u1,u2,u3,v1,w,v2,x,y,theta, Lambda, alpha1, alpha2, alpha3,  X[0],Y[0],THETA[0], X[-1],Y[-1],THETA[-1]) <= 10**(-6))
    opti.subject_to( x[1:] + 10**(-4) == x[:n-1]+taux*(cos(theta[:n-1])*v1[:n-1] - sin(theta[:n-1])*v2[:n-1]) )
    opti.subject_to( y[1:] + 10**(-4) == y[:n-1]+taux*(sin(theta[:n-1])*v1[:n-1] + cos(theta[:n-1])*v2[:n-1]) )
    opti.subject_to( theta[1:]  + 10**(-4) == theta[:n-1] + taux*w[:n-1] )
    opti.subject_to( (v1[:n-1] + taux* u1[:n-1] == v1[1:] + 10**(-4))  )
    opti.subject_to( (v2[:n-1] + taux* u3[:n-1] == v2[1:] + 10**(-4)) )
    opti.subject_to( (w[:n-1] + taux* u2[:n-1] == w[1:] + 10**(-4)) )


        ## pour les conditions finales
    opti.subject_to( x[-1]==X[-1] + 10**(-4))
    opti.subject_to( y[-1]==Y[-1] + 10**(-4))
    opti.subject_to( theta[-1]==THETA[-1] + 10**(-4))

    
    opti.set_initial(alpha1, Alpha1)
    opti.set_initial(alpha2, Alpha2)
    opti.set_initial(alpha3, Alpha3)
    
    
    opti.set_initial(u1, U1)
    opti.set_initial(u2, U2)
    opti.set_initial(u3, U3)
    opti.set_initial(v1, V1)
    opti.set_initial(w, W)
    opti.set_initial(v2, V2)
    opti.set_initial(x, X)
    opti.set_initial(y, Y)
    opti.set_initial(theta, THETA)
    

    opti.solver('ipopt')    


    sol = opti.solve()
    
    return sol.value(alpha1), sol.value(alpha2), sol.value(alpha3), sol.value(x), sol.value(y), sol.value(theta)


############################### PDFO

options = {'maxfev': 400  , 'rhoend' : 1e-6}

bounds1 = np.array([[0, 1], [0, 1] , [0, 1]])
lin_con1 = LinearConstraint([1, 1, 1], 1, 1)


def Mombaur (alpha) :
    (alpha1, alpha2, alpha3) = alpha
    print(alpha)
    
    opti = casadi.Opti()   # cette fonction nous permet de trouver la solution de problème

    ## les positions
    x = opti.variable(n)
    y = opti.variable(n)
    theta = opti.variable(n)

    ## les vitesses 
    v1 = opti.variable(n)        ## vitesse latérale
    v2 = opti.variable(n)        ## vitesse orthogonal
    w = opti.variable(n)         ## vitesse angulaire


    ## les accélération 
    u1 = opti.variable(n)        ## accélération latérale
    u3 = opti.variable(n)        ## accélération orthogonal
    u2 = opti.variable(n)        ## accélération angulaire


    opti.minimize(  taux*(alpha1 * dot(u1,u1) + alpha2 * dot(u2,u2 ) + alpha3 * dot( u3 ,u3 ) ) )    # ma fonction objetion

        # mes fonctions de contrainte d'égalité:

        ## pour les condition initial
    opti.subject_to( x[0] == X1[0] + 10**(-4))        
    opti.subject_to( y[0] == X2[0] + 10**(-4))
    opti.subject_to( theta[0] == THETA[0] + 10**(-4))
    
    opti.subject_to( v1[0] == 0.0001 )
    opti.subject_to( w[0]  == 0.0001 )
    opti.subject_to( v2[0] == 0.0001 )
    opti.subject_to( v1[-1] == 0.0001 )
    opti.subject_to( w[-1]  == 0.0001 )
    opti.subject_to( v2[-1] == 0.0001 )
    
    
    
    for j in range (n): 
        opti.subject_to( (u1[j]) <= 20 )
        opti.subject_to( (u1[j]) >= -20 )
        
        opti.subject_to( (u2[j]) <= 20 )
        opti.subject_to( (u2[j]) >= -20 )
        
        opti.subject_to( (u3[j]) <= 20 )
        opti.subject_to( (u3[j]) >= -20 )
        

        
    opti.subject_to( u1[-1] == 0.0001 )
    opti.subject_to( u2[-1] == 0.0001 )
    opti.subject_to( u3[-1] == 0.0001 )

    opti.subject_to( u1[0] == 0.0001 )
    opti.subject_to( u2[0] == 0.0001 )
    opti.subject_to( u3[0] == 0.0001 )



    ## pour les contraintes d'égaliter
    opti.subject_to( x[1:] + 10**(-4) == x[:n-1]+taux*(cos(theta[:n-1])*v1[:n-1] - sin(theta[:n-1])*v2[:n-1]) )
    opti.subject_to( y[1:] + 10**(-4) == y[:n-1]+taux*(sin(theta[:n-1])*v1[:n-1] + cos(theta[:n-1])*v2[:n-1]) )
    opti.subject_to( theta[1:] + 10**(-4) == theta[:n-1] + taux*w[:n-1] )
    opti.subject_to( (v1[:n-1] + taux* u1[:n-1] == v1[1:] + 10**(-4))  )
    opti.subject_to( (v2[:n-1] + taux* u3[:n-1] == v2[1:] + 10**(-4)) )
    opti.subject_to( (w[:n-1] + taux* u2[:n-1] == w[1:] + 10**(-4)) )


        ## pour les conditions finales
    opti.subject_to( x[-1]==X1[-1] + 10**(-4))
    opti.subject_to( y[-1]==X2[-1] + 10**(-4))
    opti.subject_to( theta[-1]==THETA[-1] + 10**(-4))


    opti.solver('ipopt')      # suivant la méthode de KKT

    sol = opti.solve()
    
    X1_1 = sol.value(x)
    X2_1 = sol.value(y)
    X3_1 = sol.value(theta)
    
    m1 = sqrt((np.linalg.norm(X1-X1_1)**2 + np.linalg.norm(X2-X2_1)**2 + np.linalg.norm(THETA-X3_1)**2 )/n)

    
    return  m1   


def DOC_Mombaur_PDFO (alpha1,alpha2,alpha3):
    opti = casadi.Opti()   # cette fonction nous permet de trouver la solution de problème

    ## les positions
    x = opti.variable(n)
    y = opti.variable(n)
    theta = opti.variable(n)

    ## les vitesses 
    v1 = opti.variable(n)        ## vitesse latérale
    v2 = opti.variable(n)        ## vitesse orthogonal
    w = opti.variable(n)         ## vitesse angulaire


    ## les accélération 
    u1 = opti.variable(n)        ## accélération latérale
    u3 = opti.variable(n)        ## accélération orthogonal
    u2 = opti.variable(n)        ## accélération angulaire


    opti.minimize(  taux*(alpha1 * dot(u1,u1) + alpha2 * dot(u2,u2 ) + alpha3 * dot( u3 ,u3 ) ) )    # ma fonction objetion

        # mes fonctions de contrainte d'égalité:

        ## pour les condition initial
    opti.subject_to( x[0] == X1[0] + 10**(-4))        
    opti.subject_to( y[0] == X2[0] + 10**(-4))
    opti.subject_to( theta[0] == THETA[0] + 10**(-4))
    
    opti.subject_to( v1[0] == 0.0001 )
    opti.subject_to( w[0]  == 0.0001 )
    opti.subject_to( v2[0] == 0.0001 )
    opti.subject_to( v1[-1] == 0.0001 )
    opti.subject_to( w[-1]  == 0.0001 )
    opti.subject_to( v2[-1] == 0.0001 )
    
    
    
    for j in range (n): 
        opti.subject_to( (u1[j]) <= 20 )
        opti.subject_to( (u1[j]) >= -20 )
        
        opti.subject_to( (u2[j]) <= 20 )
        opti.subject_to( (u2[j]) >= -20 )
        
        opti.subject_to( (u3[j]) <= 20 )
        opti.subject_to( (u3[j]) >= -20 )
        

        
    opti.subject_to( u1[-1] == 0.0001 )
    opti.subject_to( u2[-1] == 0.0001 )
    opti.subject_to( u3[-1] == 0.0001 )

    opti.subject_to( u1[0] == 0.0001 )
    opti.subject_to( u2[0] == 0.0001 )
    opti.subject_to( u3[0] == 0.0001 )



    ## pour les contraintes d'égaliter
    opti.subject_to( x[1:] + 10**(-4) == x[:n-1]+taux*(cos(theta[:n-1])*v1[:n-1] - sin(theta[:n-1])*v2[:n-1]) )
    opti.subject_to( y[1:] + 10**(-4) == y[:n-1]+taux*(sin(theta[:n-1])*v1[:n-1] + cos(theta[:n-1])*v2[:n-1]) )
    opti.subject_to( theta[1:] + 10**(-4) == theta[:n-1] + taux*w[:n-1] )
    opti.subject_to( (v1[:n-1] + taux* u1[:n-1] == v1[1:] + 10**(-4))  )
    opti.subject_to( (v2[:n-1] + taux* u3[:n-1] == v2[1:] + 10**(-4)) )
    opti.subject_to( (w[:n-1] + taux* u2[:n-1] == w[1:] + 10**(-4)) )


        ## pour les conditions finales
    opti.subject_to( x[-1]==X1[-1] + 10**(-4))
    opti.subject_to( y[-1]==X2[-1] + 10**(-4))
    opti.subject_to( theta[-1]==THETA[-1] + 10**(-4))


    opti.solver('ipopt')      # suivant la méthode de KKT

    sol = opti.solve()
    
    X1_1 = sol.value(x)
    X2_1 = sol.value(y)
    X3_1 = sol.value(theta)
    
    return X1_1, X2_1,X3_1

In [5]:
i = 0

T1 = np.loadtxt("human_traj_3.dat")
X1 = T1[0+6*i]
X2 = T1[1+6*i]
X3 = atan(T1[3+6*i]/T1[2+6*i])

M00 = vertcat(0,X3[1:])


Xi0 = [X1[0],X2[0],X3[0]]
Xf0 = [X1[-1],X2[-1],X3[-1]]

U1 = cos(X3)*T1[2+6*i]+T1[3+6*i]*sin(X3)
U2 = (M00-X3)/taux


X = T1[0+6*i]
Y = T1[1+6*i]
THETA = T1[5+6*i]
M0 = vertcat(0,THETA[1:])

Xi = [X[0],Y[0],THETA[0]]
Xf = [X[-1],Y[-1],THETA[-1]]

V1 = T1[2+6*i]*cos(THETA) + sin(THETA)*T1[3+6*i]
V2 = -T1[2+6*i]*sin(THETA) + cos(THETA)*T1[3+6*i]
W  = (M0-T1[5+6*i])/taux

M1 = vertcat(V1[1:],0)
M2 = vertcat(W[1:],0)
M3 = vertcat(V2[1:],0)

U1_1 = (M1-V1)/taux
U2_1 = (M2-W)/taux
U3_1 = (M3-V2)/taux

In [ ]:
mean_traj1_rmse_plan = np.zeros(6)
mean_traj1_rmse_ang_rad = np.zeros(6)
mean_traj1_rmse_ang_degré = np.zeros(6)

## KKT

alpha1, alpha2, alpha3, Lambda = Katja_Mombaur_IOC (R_K,U1_1,U2_1,U3_1, V1,V2,W,X,Y,THETA,Xi,Xf)

X1M_KKT,X2M_KKT,X3M_KKT, V1_S1,V2_S1,W_S1, U1_S1,U2_S1,U3_S1 = Katja_Mombaur_DOC (Xi, Xf, alpha1, alpha2, alpha3)  
    
mean_traj1_rmse_plan[0] = sqrt((dot(X-X1M_KKT, X-X1M_KKT ) + dot(Y-X2M_KKT, Y-X2M_KKT ))/n)
mean_traj1_rmse_ang_rad[0] = sqrt((dot(THETA-X3M_KKT, THETA-X3M_KKT ) )/n)
mean_traj1_rmse_ang_degré[0] = sqrt((dot((THETA-X3M_KKT)*180/pi, (THETA-X3M_KKT)*180/pi ) )/n)


## PDFO 

res = pdfo( Mombaur, [0.1 ,0.5, 0.4], bounds=bounds1, constraints=[lin_con1], options=options)
alpha1,alpha2,alpha3 = res.x
X1M_PDFO,X2M_PDFO,X3M_PDFO = DOC_Mombaur_PDFO (alpha1,alpha2,alpha3)

mean_traj1_rmse_plan[1] = sqrt((dot(X-X1M_PDFO, X-X1M_PDFO ) + dot(Y-X2M_PDFO, Y-X2M_PDFO ))/n)
mean_traj1_rmse_ang_rad[1] = sqrt((dot(THETA-X3M_PDFO, THETA-X3M_PDFO ) )/n)
mean_traj1_rmse_ang_degré[1] = sqrt((dot((THETA-X3M_PDFO)*180/pi, (THETA-X3M_PDFO)*180/pi ) )/n)

##  BL1

alpha1,alpha2,alpha3, X1M_BL1,X2M_BL1,X3M_BL1   = BL_Mombaur (U1_1,U2_1,U3_1 ,V1,W,V2,X,Y,THETA, 0.25,0.5,0.25)
X1M_BL1,X2M_BL1,X3M_BL1, V1_S1,V2_S1,W_S1, U1_S1,U2_S1,U3_S1 = Katja_Mombaur_DOC (Xi, Xf, alpha1, alpha2, alpha3)  


mean_traj1_rmse_plan[2] = sqrt((dot(X-X1M_BL1, X-X1M_BL1 ) + dot(Y-X2M_BL1, Y-X2M_BL1 ))/n)
mean_traj1_rmse_ang_rad[2] = sqrt((dot(THETA-X3M_BL1, THETA-X3M_BL1 ) )/n)
mean_traj1_rmse_ang_degré[2] = sqrt((dot((THETA-X3M_BL1)*180/pi, (THETA-X3M_BL1)*180/pi ) )/n)

############ Modèle non-holonomique 

## KKT

c1,c2, Lambda, Mue = Unicycle_IOC (R_U, U1,U2,X1,X2,X3,Xi0,Xf0)
X1P_KKT,X2P_KKT,X3P_KKT, U1_S1,U2_S1 = Unicycle_DOC ( Xi0 , Xf0 , c1 , c2)

mean_traj1_rmse_plan[3] = sqrt((dot(X-X1P_KKT, X-X1P_KKT ) + dot(Y-X2P_KKT, Y-X2P_KKT ))/n)
mean_traj1_rmse_ang_rad[3] = sqrt((dot(X3-X3P_KKT, X3-X3P_KKT ) )/n)
mean_traj1_rmse_ang_degré[3] = sqrt((dot((X3-X3P_KKT)*180/pi, (X3-X3P_KKT)*180/pi ) )/n)

## PDFO 

res = pdfo( Unicycle, [0.5, 0.5], bounds=bounds, constraints=[lin_con], options=options)
c1,c2= res.x

X1P_PDFO,X2P_PDFO,X3P_PDFO = DOC_Unicycle_PDFO (Xi0, Xf0, c1,c2)

mean_traj1_rmse_plan[4] = sqrt((dot(X-X1P_PDFO, X-X1P_PDFO ) + dot(Y-X2P_PDFO, Y-X2P_PDFO ))/n)
mean_traj1_rmse_ang_rad[4] = sqrt((dot(X3-X3P_PDFO, X3-X3P_PDFO ) )/n)
mean_traj1_rmse_ang_degré[4] = sqrt((dot((X3-X3P_PDFO)*180/pi, (X3-X3P_PDFO)*180/pi ) )/n)

##  BL1

c1,c2, X1P_BL1,X2P_BL1,X3P_BL1   = BL (U1,U2,X1,X2,X3,0.5,0.5,Xi0, Xf0)
X1P_BL1,X2P_BL1,X3P_BL1, U1_S1,U2_S1 = Unicycle_DOC ( Xi0 , Xf0 , c1 , c2)

mean_traj1_rmse_plan[5] = sqrt((dot(X-X1P_BL1, X-X1P_BL1 ) + dot(Y-X2P_BL1, Y-X2P_BL1 ))/n)
mean_traj1_rmse_ang_rad[5] = sqrt((dot(X3-X3P_BL1, X3-X3P_BL1 ) )/n)
mean_traj1_rmse_ang_degré[5] = sqrt((dot((X3-X3P_BL1)*180/pi, (X3-X3P_BL1)*180/pi ) )/n)

In [ ]:
plt.figure(figsize = (15,8))
plt.subplot(1,2,1)
plt.title("Modèle holonomique \n Trajctoire moyenne 3")
plt.plot(X,Y, color = 'r', label = 'traj moyenne')
plt.plot(X1M_KKT,X2M_KKT, color = 'b', label = 'KKT')
plt.plot(X1M_PDFO,X2M_PDFO, color = 'green', label = 'PDFO')
plt.plot(X1M_BL1,X2M_BL1, color = 'yellow', label = 'Bi-level en un coup')
plt.xlabel("X [m]")
plt.ylabel("Y [m]")
plt.legend()

plt.subplot(1,2,2)
plt.title("Modèle non-holonomique \n Trajctoire moyenne 3")
plt.plot(X,Y, color = 'r', label = 'traj moyenne')
plt.plot(X1P_KKT,X2P_KKT, color = 'b', label = 'KKT')
plt.plot(X1P_PDFO,X2P_PDFO, color = 'green', label = 'PDFO')
plt.plot(X1P_BL1,X2P_BL1, color = 'yellow', label = 'Bi-level en un coup')
plt.xlabel("X [m]")
plt.ylabel("Y [m]")
plt.legend()

In [ ]:
df = pd.DataFrame({'Trajectoire moyenne' : ['KKT','PDFO', 'Bi-level en un coup','KKT','PDFO', 'Bi-level en un coup'],
                   'Trajectoire_3_RMSE_du_plan_(x,y) unité [m]' : mean_traj1_rmse_plan, 'Trajectoire_1_RMSE_angulaire unité [rad]' : mean_traj1_rmse_ang_rad, 'Trajectoire_1_RMSE_angulaire unité [°]' : mean_traj1_rmse_ang_degré}, 
                 index = ['Modèle holonomique','Modèle holonomique','Modèle holonomique','Modèle non-holonomique','Modèle non-holonomique','Modèle non-holonomique'])

df.to_csv('traj3mean.csv', index = True)
traj3_mean = pd.read_csv("traj3mean.csv")
dfi.export(traj3_mean, 'traj3_mean.png')